In [1]:
# Imports
import os
import pandas as pd
from importlib.metadata import files
import numpy as np

In [27]:
# Read in the data
directory = '../data/'
extension = '.csv'
columns = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']

df = pd.DataFrame()

# Iterate through all sub-directories and files
for dirpath, dirnames, files in os.walk(directory):
    for file in files:
        if file.endswith(extension):
            # Open the file and read its contents
            contents = pd.read_csv(os.path.join(dirpath, file))
            df = df.append(contents[columns])

df.head()

/var/folders/7y/v7whc3tn0fl5rddh_2ql4n3m0000gn/T/ipykernel_5430/3726563229.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(contents[columns])
/var/folders/7y/v7whc3tn0fl5rddh_2ql4n3m0000gn/T/ipykernel_5430/3726563229.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(contents[columns])
/var/folders/7y/v7whc3tn0fl5rddh_2ql4n3m0000gn/T/ipykernel_5430/3726563229.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(contents[columns])
/var/folders/7y/v7whc3tn0fl5rddh_2ql4n3m0000gn/T/ipykernel_5430/3726563229.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(contents[column

,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,Fulham,Arsenal,0,3,A
1,Crystal Palace,Southampton,1,0,H
2,Liverpool,Leeds,4,3,H
3,West Ham,Newcastle,0,2,A
4,West Brom,Leicester,0,3,A
